install all the packages:

In [44]:
import pandas as pd
import numpy as np
from adtoolbox import core,configs,utils,adm
import plotly.express as px

In [2]:
acids=pd.read_table('../raw_data/DeLong_beto/acids.csv',delimiter=',')
metadata=pd.read_table("/Users/parsaghadermarzi/Desktop/Academics/Projects/ADSimulations/raw_data/DeLong_beto/DeLong_metadata_final[72].txt", delimiter="\t")

Filter/delete duplicates on the acid column

In [3]:
acids2adm={
    "C4":"S_bu",
    "C3":"S_pro",
    "C5":"S_va",
    "ISOC5":"S_va",
    "ISOC4":"S_bu",
    "C2":"S_ac",
    "C6":"S_cap",
    "ISOC6":"S_cap"

}

In [4]:
def identifier(items):
    if items in acids2adm:
        return acids2adm[items]

acids["admids"]=acids["acid"].apply(identifier)

In [6]:
acids2seed={
    "C4":"cpd00211",
    "C3":"cpd00141",
    "C5":"cpd00597",
    "ISOC5":"cpd00597",
    "ISOC4":"cpd00211",
    "C2":"cpd00029",
    "C6":"cpd01113",
    "ISOC6":"cpd01113"

}

In [7]:
def identity(items):
    if items in acids2seed:
        return acids2seed[items]

acids["seedids"]=acids["acid"].apply(identity)

We have to make sure we have Seed databases downloaded for COD calculations

In [10]:
from adtoolbox import core,configs
seed_db=core.SeedDB(configs.Database())
db=core.Database(configs.Database())
# db.download_seed_databases()

In [11]:
acids=acids.dropna()

In [12]:
codmap={ i:seed_db.instantiate_metabs(i).cod_calc(add_h=1) for i in acids["seedids"].unique()}

In [13]:
acids["cod_map"]=acids["seedids"].apply(lambda x:codmap[x])

In [14]:
acids['cod_conc']=acids['conc_gL']*acids['cod_map']

In [15]:
acids["shortsampleid"]=(acids.sample_id.str.split('_').apply(lambda x:"_".join(x[:5])))


We don't want to simulate for Control and inoculum only samples

In [18]:
acids=acids[~acids["rep"].str.contains("INOC")]
acids=acids[~acids["feedstock"].str.contains("Control")]

In [19]:
pivot_df = acids.pivot_table(index=["shortsampleid", "day"], columns="acid", values="cod_conc", fill_value=0)
pivot_df["C4"]=pivot_df["C4"]+pivot_df["ISOC4"]
pivot_df["C5"]=pivot_df["C5"]+pivot_df["ISOC5"]
pivot_df["C6"]=pivot_df["C6"]+pivot_df["ISOC6"]
pivot_df=pivot_df.drop(columns=["ISOC4","ISOC5","ISOC6"])
pivot_df.rename(columns={"C4":"S_bu","C3":"S_pro","C5":"S_va","C2":"S_ac","C6":"S_cap"},inplace=True)
data_dict = {}
for reactor in pivot_df.index.levels[0]:
  data_dict[reactor] = {}
  data_dict[reactor]["time"]=pivot_df.loc[reactor].index.tolist()
  data_dict[reactor]["data"]=pivot_df.loc[reactor].to_dict(orient="list")


In [21]:
cod_counts=pd.read_csv('./COD_counts.csv')

In [22]:
data_i={}
for i in cod_counts.columns[cod_counts.columns.str.endswith("_D0")]:
  data_i[i]=cod_counts.set_index("Unnamed: 0")[i].to_dict()



In [23]:
pivot_df = acids.pivot_table(index=["shortsampleid", "day"], columns="acid", values="cod_conc", fill_value=0)
pivot_df["C4"]=pivot_df["C4"]+pivot_df["ISOC4"]
pivot_df["C5"]=pivot_df["C5"]+pivot_df["ISOC5"]
pivot_df["C6"]=pivot_df["C6"]+pivot_df["ISOC6"]
pivot_df=pivot_df.drop(columns=["ISOC4","ISOC5","ISOC6"])
pivot_df.rename(columns={"C4":"S_bu","C3":"S_pro","C5":"S_va","C2":"S_ac","C6":"S_cap"},inplace=True)
data_dict = {}
for reactor in pivot_df.index.levels[0]:
  data_dict[reactor] = {}
  data_dict[reactor]["time"]=pivot_df.loc[reactor].index.tolist()
  data_dict[reactor]["data"]=pivot_df.loc[reactor].to_dict(orient="list")

  try:
    data_dict[reactor]["Initial conditions"]=cod_counts.set_index("Unnamed: 0")[reactor+"_D0"].to_dict()
  except:
    pass

In [24]:
feed_manure = metadata[metadata['feedstock']==('M')]

In [25]:
feed_manure=feed_manure['feedstock']

In [26]:
feed_waste = metadata[metadata['feedstock']==('F')]

In [27]:
feed_waste=feed_waste['feedstock']

In [28]:
manure = core.Feed(
        name="Manure",
        lipids=10.7,
        proteins=17.3,
        carbohydrates=70,
        xi= 60,
        si=50,
        tss=80,
)

In [29]:
foodwaste = core.Feed(
        name="foodwaste",
        lipids=21,
        proteins=20.5,
        carbohydrates=50.1,
        xi=60,
        si=50,
        tss=65
)

In [30]:
core.Database(configs.Database(
    adm_parameters_urls=configs.E_ADM_2_REMOTE,
    adm_parameters=configs.E_ADM_2_LOCAL
)).download_adm_parameters()


model_parameters downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramet
ers/e_adm_2_model_parameters.json

base_parameters downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramete
rs/e_adm_2_base_parameters.json

initial_conditions downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Param
eters/e_adm_2_initial_conditions.json

inlet_conditions downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramet
ers/e_adm_2_inlet_conditions.json

reactions downloaded to 
/Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Parameters/e_adm_2_reactions.json

species downloaded to 
/Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Parameters/e_adm_2_species.json

In [35]:
params=utils.load_multiple_json_files(configs.E_ADM_2_LOCAL)

In [36]:
Manure = core.Feed(
        name="Manure",
        lipids=10.7,
        proteins=30,
        carbohydrates=59.3,
        xi= 60,
        si=50,
        tss=80,
 )
foodwaste = core.Feed(
        name="foodwaste",
        lipids=21,
        proteins=20.5,
        carbohydrates=50.1,
        xi=60,
        si=50,
        tss=65
)

In [37]:
results_dict = {}
# dash_apps = {}
inoculum_cod=10
total_feed_cod=15
params.base_parameters['q_in'] = 0
for reactor_name, reactor_data in data_dict.items():
    if "Initial conditions" in reactor_data:
        initial_conditions = {key:value*inoculum_cod for key,value in reactor_data["Initial conditions"].items()}
        initial_conditions["X_VFA_deg"]=0.1*inoculum_cod
        _, t, f, ph, _ = reactor_name.split("_")
        print(float(t), f, float(ph))

        if f == "F":
            feed = foodwaste
        else:
            feed = Manure
        initial_conditions["TSS"]=total_feed_cod*feed.tss/100
        initial_conditions["TDS"]=total_feed_cod-initial_conditions["TSS"]
        initial_conditions["s_su"]=0
        initial_conditions["s_aa"]=0
        initial_conditions["s_va"]=0
        initial_conditions["s_fa"]=0
        initial_conditions["s_ac"]=0
        initial_conditions["s_cap"]=0
        params.initial_conditions.update(initial_conditions)
        model = adm.Model(
            model_parameters=params.model_parameters,
            base_parameters=params.base_parameters,
            initial_conditions=params.initial_conditions,
            inlet_conditions=params.inlet_conditions,
            species=params.species,
            reactions=params.reactions,
            feed=feed,
            ode_system=adm.e_adm_2_ode_sys,
            build_stoichiometric_matrix=adm.build_e_adm_2_stoichiometric_matrix,
            time_limit=100000,
            control_state={'S_H_ion': 10 ** -(float(ph))})

        sol = model.solve_model(np.linspace(0, 20, 10))
        results_dict[reactor_name] = pd.DataFrame(sol.y.T,columns=params.species)



35.0 F 5.0
35.0 F 5.0
35.0 F 7.0
35.0 F 7.0
35.0 F 7.0
35.0 F 9.0
35.0 F 9.0
35.0 F 9.0
35.0 M 5.0
35.0 M 5.0
35.0 M 7.0
35.0 M 9.0
35.0 M 9.0
35.0 M 9.0
45.0 F 7.0
45.0 F 7.0
45.0 F 7.0
45.0 F 9.0
45.0 F 9.0
45.0 F 9.0
45.0 M 5.0
45.0 M 7.0
45.0 M 7.0
45.0 M 7.0
35.0 F 5.0
35.0 F 5.0
35.0 F 5.0
35.0 F 7.0
35.0 F 7.0
35.0 F 7.0
35.0 F 9.0
35.0 F 9.0
35.0 F 9.0
35.0 M 5.0
35.0 M 5.0
35.0 M 5.0
35.0 M 7.0
35.0 M 7.0
35.0 M 7.0
35.0 M 9.0
35.0 M 9.0
45.0 F 5.0
45.0 F 7.0
45.0 F 9.0
45.0 M 5.0
45.0 M 5.0
45.0 M 5.0
45.0 M 7.0
45.0 M 9.0


In [38]:
only_initial_conditions = {}
for data, initial in data_dict.items():
    if "Initial conditions" in initial:
        only_initial_conditions[data] = pd.DataFrame(initial["data"]).max()

In [39]:
exp_data_sim=pd.DataFrame(only_initial_conditions).T

In [40]:
exp_data_sim["Feed"]=exp_data_sim.index.map(lambda x:x.split("_")[2])

In [41]:
exp_data_sim["PH"]=exp_data_sim.index.map(lambda x:x.split("_")[3])

In [46]:
fig=px.box(exp_data_sim,x="Feed",y="S_bu",facet_col="PH",color="Feed",points="all")
fig.update_layout(title="Butyric Acid Concentration Experimental Data")
fig.write_html("Butyric Acid Concentratio Experimental Data.html")
fig.show()

In [47]:
observed_acids=exp_data_sim.columns[:-2].to_list()
for reactor in results_dict:
  results_dict[reactor]=results_dict[reactor][observed_acids].max()


In [48]:
simulations_vfa_results=pd.DataFrame(results_dict).T

In [51]:
simulations_vfa_results["Feed"]=simulations_vfa_results.index.map(lambda x:x.split("_")[2])

In [52]:
simulations_vfa_results["PH"]=simulations_vfa_results.index.map(lambda x:x.split("_")[3])

In [53]:
simulations_vfa_results

,S_ac,S_pro,S_bu,S_va,S_cap,Feed,PH
A_35_F_5_A,1.303548,2.761579,1.120776,2.456817,0.193297,F,5
A_35_F_5_C,1.309029,2.763502,1.124501,2.467845,0.194127,F,5
A_35_F_7_A,1.388743,2.740308,1.042940,2.501882,0.184603,F,7
A_35_F_7_B,1.433926,2.723645,1.091925,2.635407,0.194383,F,7
A_35_F_7_C,1.447960,2.721361,1.106966,2.672394,0.196978,F,7
A_35_F_9_A,2.697865,1.702730,0.337708,3.491567,0.224463,F,9
A_35_F_9_B,2.701234,1.699148,0.338081,3.498390,0.224656,F,9
A_35_F_9_C,2.712273,1.694214,0.338640,3.516200,0.225048,F,9
A_35_M_5_B,1.187267,2.440341,1.034827,2.461358,0.185310,M,5
A_35_M_5_C,1.187500,2.430450,1.034319,2.458191,0.184758,M,5


In [54]:
fig=px.box(simulations_vfa_results,x="Feed",y="S_bu",facet_col="PH",color="Feed",points="all")
fig.update_layout(title="Simulated Butyric Acid concentrations")
fig.write_html("simulated_vfa_concentrations.html")
fig.show()